In [38]:
import pandas as pd
import numpy as np
import warnings
import time
import itertools
import copy
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth',100)

In [39]:
dataset_name = "Music"

# 1. Read Data

In [40]:
item_df = pd.read_csv('../Dataset/Amazon_CDs_and_Vinyl/Amazon_CDs_and_Vinyl.item',sep="\t",encoding="latin-1")
item_df.columns = ["itemid","title","categories","brand","sales_type","sales_rank"]#,"price"
item_df.shape[0]
item_df = item_df[item_df["sales_type"]=="CDs & Vinyl"].reset_index(drop=True)
item_df
(item_df.isna().sum()/item_df.shape[0]).sort_values(ascending=False)

412325

,itemid,title,categories,brand,sales_type,sales_rank
0,0001393774,Songs for the Shepherd,'Christian',Keith Green,CDs & Vinyl,123896.0
1,0001515209,I Love You,'Christian',Integrity Music,CDs & Vinyl,1138833.0
2,0005072298,Hymns: 16 Classic Hymns for Children,'Christian',Sue Gay,CDs & Vinyl,271064.0
3,0005121515,Christmas Carols,'Christian',Cedarmont Kids,CDs & Vinyl,409293.0
4,0005164885,Christmas Eve and Other Stories,NaN,Trans-Siberian Orchestra,CDs & Vinyl,73.0
...,...,...,...,...,...,...
400310,B01HJ7FPAG,Furnaces,'Pop',Ed Harcourt,CDs & Vinyl,251328.0
400311,B01HJDG28E,Night Hawk,'Country',Charlie Daniels Band,CDs & Vinyl,68288.0
400312,B01HJG3UQI,Original Album Classics,"'Pop', 'Vocal Pop'",Dionne Warwick,CDs & Vinyl,69386.0
400313,B01HJG3VZI,Original Album Classics,'Pop',The Guess Who,CDs & Vinyl,1795.0


categories    0.031078
brand         0.008811
title         0.000185
sales_rank    0.000055
itemid        0.000000
sales_type    0.000000
dtype: float64

In [41]:
item_df.drop(["sales_rank"], axis=1, inplace=True)
item_df = item_df.dropna().reset_index(drop=True) 
item_df["title"] = item_df["title"].apply(lambda x: "-1" if str(x)[0]=="<" else str(x))
item_df = item_df[item_df["title"]!="-1"].reset_index(drop=True)
item_df

,itemid,title,categories,brand,sales_type
0,0001393774,Songs for the Shepherd,'Christian',Keith Green,CDs & Vinyl
1,0001515209,I Love You,'Christian',Integrity Music,CDs & Vinyl
2,0005072298,Hymns: 16 Classic Hymns for Children,'Christian',Sue Gay,CDs & Vinyl
3,0005121515,Christmas Carols,'Christian',Cedarmont Kids,CDs & Vinyl
4,0005465079,Forgiven,'Christian',Don Francisco,CDs & Vinyl
...,...,...,...,...,...
373990,B01HJ7FPAG,Furnaces,'Pop',Ed Harcourt,CDs & Vinyl
373991,B01HJDG28E,Night Hawk,'Country',Charlie Daniels Band,CDs & Vinyl
373992,B01HJG3UQI,Original Album Classics,"'Pop', 'Vocal Pop'",Dionne Warwick,CDs & Vinyl
373993,B01HJG3VZI,Original Album Classics,'Pop',The Guess Who,CDs & Vinyl


In [42]:
rating_df = pd.read_csv('../Dataset/Amazon_CDs_and_Vinyl/Amazon_CDs_and_Vinyl.inter',sep="\t")
rating_df.columns = ["userid","itemid","rating","timestamp"]
rating_df = rating_df[rating_df["itemid"].isin(item_df.itemid.tolist())].reset_index(drop=True) # fliter no title
rating_df
(rating_df.isna().sum()/rating_df.shape[0]).sort_values(ascending=False)
rating_df["userid"].nunique()
rating_df["itemid"].nunique()

,userid,itemid,rating,timestamp
0,A171I27YBM4FL6,0001393774,5.0,1461888000
1,A1H1DL4K669VQ9,0001393774,5.0,1461888000
2,A23WIHT5886G36,0001393774,5.0,1461024000
3,A3SZNOJP8OL26X,0001393774,5.0,1459296000
4,A3V5XBBT7OZG5G,0001393774,5.0,1456185600
...,...,...,...,...
3751424,A1KCWOF28NPM3M,B01HJG3VZI,5.0,1509753600
3751425,ASOJC2B605GDG,B01HJG3UQI,5.0,1492992000
3751426,A13EWHP1W5X77J,B01HJG3UQI,4.0,1480982400
3751427,A1QN6Z2E6M0S2D,B01HJG3UQI,5.0,1476316800


userid       0.0
itemid       0.0
rating       0.0
timestamp    0.0
dtype: float64

1671438

373979

In [43]:
item_interactions_count = rating_df[["userid","itemid"]].groupby(["itemid"]).count()
item_interactions_count.columns = ["count"]
item_interactions_count.sort_values(["count"] , inplace=True, ascending=False)
item_interactions_count

,count
itemid,
B00L98V4UW,7218
B00NEJ7MMI,5065
B00SWBLS3C,3674
B000VZK7T8,3468
B00MRHANNI,3087
...,...
B000BKXC0O,1
B000BKY5XM,1
B000031WA1,1


In [44]:
item_interactions_count[:50000]

,count
itemid,
B00L98V4UW,7218
B00NEJ7MMI,5065
B00SWBLS3C,3674
B000VZK7T8,3468
B00MRHANNI,3087
...,...
B0000014TT,13
B000KCHXUI,13
B000005KWK,13


# 2. Fliter

In [45]:
item_title_exist_idset = item_interactions_count[:50000].index.tolist()
rating_df_fliter = rating_df[rating_df["itemid"].isin(item_title_exist_idset)].reset_index(drop=True)
rating_df_fliter

,userid,itemid,rating,timestamp
0,A171I27YBM4FL6,0001393774,5.0,1461888000
1,A1H1DL4K669VQ9,0001393774,5.0,1461888000
2,A23WIHT5886G36,0001393774,5.0,1461024000
3,A3SZNOJP8OL26X,0001393774,5.0,1459296000
4,A3V5XBBT7OZG5G,0001393774,5.0,1456185600
...,...,...,...,...
2712847,A2IZVZU722A437,B01HJG3VZI,5.0,1529107200
2712848,A2IZEPJDLZV0C1,B01HJG3VZI,5.0,1521676800
2712849,A1T9DD4TZJO6AP,B01HJG3VZI,5.0,1516838400
2712850,A3PM5CYOKYT1AB,B01HJG3VZI,5.0,1513296000


In [46]:
rating_df_fliter["rating"].value_counts()

5.0    1981550
4.0     402179
3.0     159578
1.0      91485
2.0      78060
Name: rating, dtype: int64

In [47]:
rating_df_fliter["label"] = rating_df_fliter["rating"].apply(lambda x: 1 if x>=4 else 0)
rating_df_fliter
rating_df_fliter[rating_df_fliter["label"]==1].shape[0] / rating_df_fliter.shape[0]

,userid,itemid,rating,timestamp,label
0,A171I27YBM4FL6,0001393774,5.0,1461888000,1
1,A1H1DL4K669VQ9,0001393774,5.0,1461888000,1
2,A23WIHT5886G36,0001393774,5.0,1461024000,1
3,A3SZNOJP8OL26X,0001393774,5.0,1459296000,1
4,A3V5XBBT7OZG5G,0001393774,5.0,1456185600,1
...,...,...,...,...,...
2712847,A2IZVZU722A437,B01HJG3VZI,5.0,1529107200,1
2712848,A2IZEPJDLZV0C1,B01HJG3VZI,5.0,1521676800,1
2712849,A1T9DD4TZJO6AP,B01HJG3VZI,5.0,1516838400,1
2712850,A3PM5CYOKYT1AB,B01HJG3VZI,5.0,1513296000,1


0.878680075433529

In [48]:
item_pop = rating_df_fliter[rating_df_fliter["label"]==1]["itemid"].value_counts().rename_axis("itemid").reset_index(name="pop")
item_pop

,itemid,pop
0,B00L98V4UW,6555
1,B00NEJ7MMI,4858
2,B00SWBLS3C,3527
3,B00KLF5J64,2808
4,B00MRHANNI,2759
...,...,...
49992,B00284BWUS,1
49993,B000QUU2N4,1
49994,B0000A0DU0,1
49995,B00284BWU8,1


In [49]:
rating_df_fliter.sort_values(["userid", "timestamp"] , inplace=True, ascending=True) 
rating_df_fliter = rating_df_fliter.reset_index(drop=True)
sequence_df = rating_df_fliter.groupby(['userid']).agg(
    itemid_seq=("itemid", list),
    label_seq=("label", list)
).reset_index()
sequence_df

,userid,itemid_seq,label_seq
0,A0001624UKLQG4OFIM8X,"[B000002KIC, B000002UDB, B000002KE5]","[1, 1, 1]"
1,A0005916MHK9RK69491E,[B000002UV9],[1]
2,A0006650PUSTDDZX7UKW,[B00991TCOG],[1]
3,A00201164H8T5GSBKYFP,"[B0000025RJ, B000001A6F, B0000025HU, B00000269M, B000002H70, B000002JS6, B000002USH, B006CVOKE8,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]"
4,A0029778N21C3FA8QGT2,[B00DOPXC94],[1]
...,...,...,...
1337064,AZZZG8PGB1FS0,[B006UFHBZ6],[1]
1337065,AZZZHA261DJ6V,"[B000002N74, B00005LLA2]","[0, 1]"
1337066,AZZZJIG4UBIF8,[B001XJNYPA],[1]
1337067,AZZZLM1E5JJ8C,"[B00000331J, B00000331F]","[1, 1]"


In [50]:
def get_seq(row):
    pos_seq = []
    neg_seq = []
    for i in range(len(row.label_seq)):
        if row.label_seq[i]==1:
            pos_seq.append(row.itemid_seq[i])
        else:
            neg_seq.append(row.itemid_seq[i])
    return pos_seq, neg_seq, len(row.itemid_seq), len(pos_seq), len(neg_seq)


sequence_df[["pos_seq","neg_seq", "seq_length", "pos_seq_length", "neg_seq_length"]] = sequence_df.apply(get_seq, axis=1, result_type="expand") 
sequence_df

,userid,itemid_seq,label_seq,pos_seq,neg_seq,seq_length,pos_seq_length,neg_seq_length
0,A0001624UKLQG4OFIM8X,"[B000002KIC, B000002UDB, B000002KE5]","[1, 1, 1]","[B000002KIC, B000002UDB, B000002KE5]",[],3,3,0
1,A0005916MHK9RK69491E,[B000002UV9],[1],[B000002UV9],[],1,1,0
2,A0006650PUSTDDZX7UKW,[B00991TCOG],[1],[B00991TCOG],[],1,1,0
3,A00201164H8T5GSBKYFP,"[B0000025RJ, B000001A6F, B0000025HU, B00000269M, B000002H70, B000002JS6, B000002USH, B006CVOKE8,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[B0000025RJ, B000001A6F, B0000025HU, B00000269M, B000002H70, B000002JS6, B000002USH, B006CVOKE8,...",[B00914JWSW],15,14,1
4,A0029778N21C3FA8QGT2,[B00DOPXC94],[1],[B00DOPXC94],[],1,1,0
...,...,...,...,...,...,...,...,...
1337064,AZZZG8PGB1FS0,[B006UFHBZ6],[1],[B006UFHBZ6],[],1,1,0
1337065,AZZZHA261DJ6V,"[B000002N74, B00005LLA2]","[0, 1]",[B00005LLA2],[B000002N74],2,1,1
1337066,AZZZJIG4UBIF8,[B001XJNYPA],[1],[B001XJNYPA],[],1,1,0
1337067,AZZZLM1E5JJ8C,"[B00000331J, B00000331F]","[1, 1]","[B00000331J, B00000331F]",[],2,2,0


In [51]:
sequence_df_fliter = sequence_df[(sequence_df["pos_seq_length"]>=6) & (sequence_df["neg_seq_length"]>=25)].reset_index(drop=True)
item_df = item_df[item_df["itemid"].isin(list(itertools.chain.from_iterable(sequence_df_fliter["itemid_seq"].values.tolist())))].reset_index(drop=True)
sequence_df_fliter
item_df

,userid,itemid_seq,label_seq,pos_seq,neg_seq,seq_length,pos_seq_length,neg_seq_length
0,A10KCGSDJJAVRO,"[B000000IRW, B000002KHL, B000002KBT, B00007L6HY, B0001LJCZ2, B00006DTZ7, B0000026JN, B0007XBN1U,...","[1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,...","[B000000IRW, B000002KHL, B00007L6HY, B0001LJCZ2, B0000026JN, B0007XBN1U, B000002I95, B0001NBLUE,...","[B000002KBT, B00006DTZ7, B000068FUS, B00007JGW9, B00006F852, B000NJXCG2, B000PKG7HU, B000ZK53CA,...",68,33,35
1,A11XKY4EIU2KNR,"[B000001FE8, B000005RFQ, B000001ELN, B000001EL3, B000001FG1, B000005RFR, B000001F4P, B000006O2T,...","[1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,...","[B000001FE8, B000005RFQ, B000001FG1, B000005RFR, B000001F4P, B000006O2T, B0000026X0, B000002JS8,...","[B000001ELN, B000001EL3, B00000I0OY, B000001ELD, B000001FPF, B00000AF98, B000001ELL, B0000025TF,...",193,135,58
2,A120UTHQDQIJGH,"[B0000035E1, B001F1YLT4, B000002LFU, B000002LOG, B0026J8LHM, B000002J0B, B000ZK4466, B000ZK4466,...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,...","[B0000035E1, B000002J0B, B003Y3BMFM, B000AYQO2O, B009ERK1T0, B000002H70, B00000F1U5, B000NJLM3C,...","[B001F1YLT4, B000002LFU, B000002LOG, B0026J8LHM, B000ZK4466, B000ZK4466, B00006AGCR, B004NXYSI0,...",63,34,29
3,A12M96JJHEOYMT,"[B00000396B, B00064X2QE, B000002I6T, B00000253N, B00000E5JL, B000Q9OD4O, B0009IE6K8, B000008GQE,...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[B00000396B, B00000253N, B00000E5JL, B000Q9OD4O, B0009IE6K8, B000025AP2, B00000117H, B000YNFY1S,...","[B00064X2QE, B000002I6T, B000008GQE, B000002OS1, B0000025BO, B0000C9ZK5, B0013FCYGE, B0013FCYGE,...",81,41,40
4,A12R54MKO17TW0,"[B0001AVYJA, B0000027GX, B000002KOD, B000002OCP, B000003493, B000002IRS, B0000032VS, B000008MIQ,...","[1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,...","[B0001AVYJA, B000002KOD, B000002OCP, B000003493, B000002IRS, B0000032VS, B0000003H1, B000001DTH,...","[B0000027GX, B000008MIQ, B0000024S9, B000002UR4, B000003BED, B000JBXOGC, B001G4JR4Y, B0000028OZ,...",715,677,38
...,...,...,...,...,...,...,...,...
409,AYOO12C9Y2T95,"[B00004TM1F, B00005JGA4, B000002LE5, B00000JJM0, B00000DPJL, B000001EG4, B000000IRG, B00002621U,...","[1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,...","[B00004TM1F, B00000DPJL, B000001EG4, B000000IRG, B00005J6Z3, B0001BFDJG, B0000024VT, B000002GKN,...","[B00005JGA4, B000002LE5, B00000JJM0, B00002621U, B000001EDN, B00061X95Q, B000002KHS, B000000WHR,...",291,119,172
410,AZD7NEVKD2GBX,"[B00000DHYK, B00004Y26G, B000002HRF, B000001FTD, B000002WYT, B000002HKK, B000050ZDO, B00001OH6I,...","[1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,...","[B00000DHYK, B00004Y26G, B000001FTD, B000002HKK, B000050HU9, B0000296JB, B00005JCRW, B00000JPM7,...","[B000002HRF, B000002WYT, B000050ZDO, B00001OH6I, B000002JWU, B00004Z45A, B000002HMH, B000002BUY,...",64,30,34
411,AZDL23124GWH3,"[B000000A1O, B000002HCK, B000002V0F, B00000K4H6, B0000026JV, B000001C9V, B000001F3K, B0000026J1,...","[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[B000000A1O, B000002HCK, B00000K4H6, B0000026JV, B000001C9V, B000001F3K, B0000026J1, B000002IMR,...","[B000002V0F, B000002HPT, B000021XS0, B000002BGM, B000007W47, B000001E1V, B000002AE3, B000002JNH,...",204,161,43
412,AZKXH0J59TK11,"[B000003J1L, B000003J21, B00000AE8M, B00006JMXH, B000002HR2, B00008ACIN, B0000AKCKR, B000003TAD,...","[0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,...","[B00008ACIN, B0000AKCKR, B0001VOQ62, B00008AWOD, B00008AWON, B000002LRJ, B000063KO8, B00005V696,...","[B000003J1L, B000003J21, B00000AE8M, B00006JMXH, B000002HR2, B000003T

,itemid,title,categories,brand,sales_type
0,0001393774,Songs for the Shepherd,'Christian',Keith Green,CDs & Vinyl
1,1858705223,Apocalyptic Love: UK Fan Pack Edition,'Rock',Slash,CDs & Vinyl
2,1858703883,Panic of Girls Fan Pack,"'Alternative Rock', 'New Wave'",Blondie,CDs & Vinyl
3,1858704553,Chickenfoot III: Classic Rock,"'Rock', 'Hard Rock'",Chickenfoot,CDs & Vinyl
4,1858705789,Re-Machined: a Tribute to Machine Head,'Rock',Deep Purple (Various Artists),CDs & Vinyl
...,...,...,...,...,...
26314,B01HEYDFJC,All the Singles,'Pop',The Turtles,CDs & Vinyl
26315,B01HF1QTPQ,Weightless,'Pop',Liz Longley,CDs & Vinyl
26316,B01HHV3R5O,Bad Moms Soundtrack,'Pop',Various,CDs & Vinyl
26317,B01HIDSTAY,Kinda Don't Care,'Country',Justin Moore,CDs & Vinyl


# 3. LabelEncoder

In [52]:
# LabelEncoder 
from sklearn import preprocessing

user_le = preprocessing.LabelEncoder()
user_le.fit(sequence_df_fliter['userid'].values.tolist())
print("user id unique nums:", len(user_le.classes_))
sequence_df_fliter['userid'] = pd.Series(user_le.transform(sequence_df_fliter['userid']))

item_le = preprocessing.LabelEncoder()
item_le.fit(list(itertools.chain.from_iterable(sequence_df_fliter["itemid_seq"].values.tolist())))
print("item id unique nums:", len(item_le.classes_))
sequence_df_fliter['pos_seq'] = sequence_df_fliter['pos_seq'].apply(lambda x: list(item_le.transform(x)))
sequence_df_fliter['neg_seq'] = sequence_df_fliter['neg_seq'].apply(lambda x: list(item_le.transform(x)))

sequence_df_fliter

LabelEncoder()

user id unique nums: 414


LabelEncoder()

item id unique nums: 26319


,userid,itemid_seq,label_seq,pos_seq,neg_seq,seq_length,pos_seq_length,neg_seq_length
0,0,"[B000000IRW, B000002KHL, B000002KBT, B00007L6HY, B0001LJCZ2, B00006DTZ7, B0000026JN, B0007XBN1U,...","[1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,...","[271, 4794, 14694, 15847, 2702, 16825, 4175, 15887, 17520, 8228, 16290, 18975, 17607, 5934, 2026...","[4688, 14260, 14079, 14679, 14281, 19151, 19332, 19943, 19943, 5471, 21238, 6898, 21975, 7666, 5...",68,33,35
1,1,"[B000001FE8, B000005RFQ, B000001ELN, B000001EL3, B000001FG1, B000005RFR, B000001F4P, B000006O2T,...","[1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,...","[1240, 8108, 1255, 8109, 1143, 8431, 2800, 4614, 9542, 10682, 1049, 5128, 8477, 3897, 6222, 1095...","[1047, 1040, 10135, 1044, 1350, 9530, 1046, 2460, 1048, 4101, 5083, 3965, 11317, 2723, 3912, 428...",193,135,58
2,2,"[B0000035E1, B001F1YLT4, B000002LFU, B000002LOG, B0026J8LHM, B000002J0B, B000ZK4466, B000ZK4466,...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,...","[6795, 4429, 22077, 17398, 23675, 3940, 9985, 19135, 24919, 282, 6948, 23899, 3770, 13408, 5153,...","[20847, 5118, 5186, 21364, 19942, 19942, 14201, 22405, 24348, 19918, 19918, 21108, 23395, 22504,...",63,34,29
3,3,"[B00000396B, B00064X2QE, B000002I6T, B00000253N, B00000E5JL, B000Q9OD4O, B0009IE6K8, B000008GQE,...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[6843, 2141, 9925, 19372, 17000, 11115, 513, 19922, 19922, 8939, 1302, 9863, 7300, 10226, 7938, ...","[16525, 4145, 9124, 5549, 2247, 15425, 20138, 20138, 15028, 4910, 20454, 2011, 4163, 5831, 20373...",81,41,40
4,4,"[B0001AVYJA, B0000027GX, B000002KOD, B000002OCP, B000003493, B000002IRS, B0000032VS, B000008MIQ,...","[1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,...","[15735, 4898, 5484, 6779, 4339, 6556, 58, 849, 6590, 6623, 8060, 8241, 6782, 5532, 5782, 6465, 6...","[2908, 9314, 2006, 5928, 6889, 18785, 20893, 3044, 4910, 5601, 21778, 9115, 1824, 2076, 2117, 42...",715,677,38
...,...,...,...,...,...,...,...,...
409,409,"[B00004TM1F, B00005JGA4, B000002LE5, B00000JJM0, B00000DPJL, B000001EG4, B000000IRG, B00002621U,...","[1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,...","[11874, 9819, 1012, 269, 12867, 15740, 2047, 3778, 339, 14258, 4407, 15442, 3448, 16386, 3360, 1...","[12900, 5099, 10676, 11146, 995, 16476, 4797, 459, 5142, 4905, 6098, 3731, 14978, 9484, 6019, 50...",291,119,172
410,410,"[B00000DHYK, B00004Y26G, B000002HRF, B000001FTD, B000002WYT, B000002HKK, B000050ZDO, B00001OH6I,...","[1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,...","[9772, 12159, 1393, 4054, 12386, 11215, 12893, 10749, 3064, 12890, 6965, 20547, 5359, 9772, 1342...","[4101, 6322, 12421, 10922, 4641, 12294, 4068, 3513, 13046, 13233, 13062, 12821, 13046, 13498, 13...",64,30,34
411,411,"[B000000A1O, B000002HCK, B000002V0F, B00000K4H6, B0000026JV, B000001C9V, B000001F3K, B0000026J1,...","[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[162, 3999, 10877, 2705, 825, 1129, 2695, 4283, 4623, 4621, 9677, 958, 1052, 1184, 1213, 4583, 3...","[5999, 4084, 11040, 3456, 8912, 919, 3280, 4571, 11211, 3403, 4038, 4042, 4039, 4352, 4351, 4585...",204,161,43
412,412,"[B000003J1L, B000003J21, B00000AE8M, B00006JMXH, B000002HR2, B00008ACIN, B0000AKCKR, B000003TAD,...","[0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,...","[14809, 15385, 15917, 14814, 14815, 5210, 13842, 13602, 3940, 16543, 15997, 3334, 14882, 13686, ...","[7195, 7196, 9511, 14449, 4094, 7382, 2339, 13459, 4031, 16302, 3353, 16119, 16544, 14089, 14098...",113,57,56


In [53]:
item_pop = item_pop[item_pop["itemid"].isin(list(item_df['itemid'].values.tolist()))].reset_index(drop=True)
item_pop["itemid"] = pd.Series(item_le.transform(item_pop['itemid']))

item_df["itemid"] = pd.Series(item_le.transform(item_df['itemid']))

In [54]:
dataset_name

'Music'

In [55]:
item_df.to_csv(f"../Dataset/{dataset_name}/processed/full_item_df.csv", sep="\t", index=False)

# 4. Gnerate Test Data for Top-K Ranking

In [56]:
def get_new_df(df, n):
    df_new = copy.deepcopy(df)
    df_new["pos_seq"] = df_new["pos_seq"].apply(lambda x: x[:-n]) 
    df_new["pos_seq_length"] = df_new["pos_seq"].apply(lambda x:len(x))

    return df_new

sequence_df_fliter["data_index"] = sequence_df_fliter["userid"].apply(lambda x: 0)
sequence_df_fliter_list = [sequence_df_fliter]
for i in range(1, 2):
    sequence_df_fliter_new = get_new_df(sequence_df_fliter, i)
    sequence_df_fliter_new["data_index"] = sequence_df_fliter_new["userid"].apply(lambda x: i)
    sequence_df_fliter_list.append(sequence_df_fliter_new)
    
full_sequence_df = pd.concat(sequence_df_fliter_list).reset_index(drop=True)
full_sequence_df

,userid,itemid_seq,label_seq,pos_seq,neg_seq,seq_length,pos_seq_length,neg_seq_length,data_index
0,0,"[B000000IRW, B000002KHL, B000002KBT, B00007L6HY, B0001LJCZ2, B00006DTZ7, B0000026JN, B0007XBN1U,...","[1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,...","[271, 4794, 14694, 15847, 2702, 16825, 4175, 15887, 17520, 8228, 16290, 18975, 17607, 5934, 2026...","[4688, 14260, 14079, 14679, 14281, 19151, 19332, 19943, 19943, 5471, 21238, 6898, 21975, 7666, 5...",68,33,35,0
1,1,"[B000001FE8, B000005RFQ, B000001ELN, B000001EL3, B000001FG1, B000005RFR, B000001F4P, B000006O2T,...","[1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1,...","[1240, 8108, 1255, 8109, 1143, 8431, 2800, 4614, 9542, 10682, 1049, 5128, 8477, 3897, 6222, 1095...","[1047, 1040, 10135, 1044, 1350, 9530, 1046, 2460, 1048, 4101, 5083, 3965, 11317, 2723, 3912, 428...",193,135,58,0
2,2,"[B0000035E1, B001F1YLT4, B000002LFU, B000002LOG, B0026J8LHM, B000002J0B, B000ZK4466, B000ZK4466,...","[1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1,...","[6795, 4429, 22077, 17398, 23675, 3940, 9985, 19135, 24919, 282, 6948, 23899, 3770, 13408, 5153,...","[20847, 5118, 5186, 21364, 19942, 19942, 14201, 22405, 24348, 19918, 19918, 21108, 23395, 22504,...",63,34,29,0
3,3,"[B00000396B, B00064X2QE, B000002I6T, B00000253N, B00000E5JL, B000Q9OD4O, B0009IE6K8, B000008GQE,...","[1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,...","[6843, 2141, 9925, 19372, 17000, 11115, 513, 19922, 19922, 8939, 1302, 9863, 7300, 10226, 7938, ...","[16525, 4145, 9124, 5549, 2247, 15425, 20138, 20138, 15028, 4910, 20454, 2011, 4163, 5831, 20373...",81,41,40,0
4,4,"[B0001AVYJA, B0000027GX, B000002KOD, B000002OCP, B000003493, B000002IRS, B0000032VS, B000008MIQ,...","[1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,...","[15735, 4898, 5484, 6779, 4339, 6556, 58, 849, 6590, 6623, 8060, 8241, 6782, 5532, 5782, 6465, 6...","[2908, 9314, 2006, 5928, 6889, 18785, 20893, 3044, 4910, 5601, 21778, 9115, 1824, 2076, 2117, 42...",715,677,38,0
...,...,...,...,...,...,...,...,...,...
823,409,"[B00004TM1F, B00005JGA4, B000002LE5, B00000JJM0, B00000DPJL, B000001EG4, B000000IRG, B00002621U,...","[1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,...","[11874, 9819, 1012, 269, 12867, 15740, 2047, 3778, 339, 14258, 4407, 15442, 3448, 16386, 3360, 1...","[12900, 5099, 10676, 11146, 995, 16476, 4797, 459, 5142, 4905, 6098, 3731, 14978, 9484, 6019, 50...",291,118,172,1
824,410,"[B00000DHYK, B00004Y26G, B000002HRF, B000001FTD, B000002WYT, B000002HKK, B000050ZDO, B00001OH6I,...","[1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,...","[9772, 12159, 1393, 4054, 12386, 11215, 12893, 10749, 3064, 12890, 6965, 20547, 5359, 9772, 1342...","[4101, 6322, 12421, 10922, 4641, 12294, 4068, 3513, 13046, 13233, 13062, 12821, 13046, 13498, 13...",64,29,34,1
825,411,"[B000000A1O, B000002HCK, B000002V0F, B00000K4H6, B0000026JV, B000001C9V, B000001F3K, B0000026J1,...","[1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,...","[162, 3999, 10877, 2705, 825, 1129, 2695, 4283, 4623, 4621, 9677, 958, 1052, 1184, 1213, 4583, 3...","[5999, 4084, 11040, 3456, 8912, 919, 3280, 4571, 11211, 3403, 4038, 4042, 4039, 4352, 4351, 4585...",204,160,43,1
826,412,"[B000003J1L, B000003J21, B00000AE8M, B00006JMXH, B000002HR2, B00008ACIN, B0000AKCKR, B000003TAD,...","[0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,...","[14809, 15385, 15917, 14814, 14815, 5210, 13842, 13602, 3940, 16543, 15997, 3334, 14882, 13686, ...","[7195, 7196, 9511, 14449, 4094, 7382, 2339, 13459, 4031, 16302, 3353, 16119, 16544, 14089, 14098...",113,56,56,1


In [57]:
import random
random.seed(2023)

def get_pos_neg_target(row):

    return row.pos_seq[-1], row.neg_seq[-1]


def get_full_candidate(row, n_candidate):
    cache = copy.deepcopy(row.neg_seq)
    cache.remove(row.neg_id_target)
    candidate = [row.pos_id_target] + [row.neg_id_target] + (random.sample(cache, n_candidate-2))
    random.shuffle(candidate)
    
    return candidate


def get_sub_candidate(row, n_candidate):
    cand_cache = copy.deepcopy(row.full_candidate)
    cand_cache.remove(row.pos_id_target)
    cand_cache.remove(row.neg_id_target) 
    candidate = [row.pos_id_target] + [row.neg_id_target] + (random.sample(list(cand_cache), n_candidate-2))
    random.shuffle(candidate)
    
    return candidate


def get_full_history(row, n_history):
    history = row.pos_seq[-1-n_history : -1]
    return history


def get_sub_history(row, n_history):
    history = row.full_history[0-n_history : ]
    return history

def get_pos_target_index(row):
    return row["itemid_candidate"].index(row["pos_id_target"])

In [59]:
full_sequence_df_new = full_sequence_df[["userid","pos_seq","neg_seq","data_index"]]
full_sequence_df_new[["pos_id_target", "neg_id_target"]] = full_sequence_df_new.apply(get_pos_neg_target, axis=1, result_type="expand")
full_sequence_df_new["full_candidate"] = full_sequence_df_new.apply(get_full_candidate, n_candidate=25, axis=1)

In [60]:
full_sequence_df_new["full_history"] = full_sequence_df_new.apply(get_full_history, n_history=25, axis=1)
full_sequence_df_new

,userid,pos_seq,neg_seq,data_index,pos_id_target,neg_id_target,full_candidate,full_history
0,0,"[271, 4794, 14694, 15847, 2702, 16825, 4175, 15887, 17520, 8228, 16290, 18975, 17607, 5934, 2026...","[4688, 14260, 14079, 14679, 14281, 19151, 19332, 19943, 19943, 5471, 21238, 6898, 21975, 7666, 5...",0,22224,7975,"[19686, 25659, 5471, 17735, 21238, 13150, 4688, 9431, 17735, 19943, 22224, 17735, 24481, 17735, ...","[15887, 17520, 8228, 16290, 18975, 17607, 5934, 20263, 20263, 21125, 4779, 21715, 17592, 22823, ..."
1,1,"[1240, 8108, 1255, 8109, 1143, 8431, 2800, 4614, 9542, 10682, 1049, 5128, 8477, 3897, 6222, 1095...","[1047, 1040, 10135, 1044, 1350, 9530, 1046, 2460, 1048, 4101, 5083, 3965, 11317, 2723, 3912, 428...",0,21638,18534,"[2723, 2460, 15616, 4101, 8359, 13186, 3965, 4369, 1044, 21638, 18534, 4286, 9530, 10643, 16758,...","[1291, 1335, 4380, 4333, 7202, 7375, 5010, 7376, 9081, 10966, 4960, 433, 264, 4774, 12207, 17956..."
2,2,"[6795, 4429, 22077, 17398, 23675, 3940, 9985, 19135, 24919, 282, 6948, 23899, 3770, 13408, 5153,...","[20847, 5118, 5186, 21364, 19942, 19942, 14201, 22405, 24348, 19918, 19918, 21108, 23395, 22504,...",0,25411,3978,"[5118, 4336, 24348, 19318, 20847, 19918, 4506, 11496, 5186, 25411, 22492, 22405, 19322, 4304, 22...","[24919, 282, 6948, 23899, 3770, 13408, 5153, 12121, 24444, 2328, 1077, 17808, 24830, 11040, 2271..."
3,3,"[6843, 2141, 9925, 19372, 17000, 11115, 513, 19922, 19922, 8939, 1302, 9863, 7300, 10226, 7938, ...","[16525, 4145, 9124, 5549, 2247, 15425, 20138, 20138, 15028, 4910, 20454, 2011, 4163, 5831, 20373...",0,22841,17148,"[1008, 1798, 15425, 20933, 4910, 2084, 20138, 21672, 11977, 22841, 20138, 9124, 9514, 5549, 6469...","[6561, 5953, 639, 2029, 9544, 18027, 4117, 11798, 12464, 13152, 1017, 6640, 2476, 10813, 13675, ..."
4,4,"[15735, 4898, 5484, 6779, 4339, 6556, 58, 849, 6590, 6623, 8060, 8241, 6782, 5532, 5782, 6465, 6...","[2908, 9314, 2006, 5928, 6889, 18785, 20893, 3044, 4910, 5601, 21778, 9115, 1824, 2076, 2117, 42...",0,22705,9644,"[22705, 4249, 6317, 21778, 2908, 9314, 13054, 9754, 3812, 4758, 2575, 2117, 20893, 3044, 3344, 6...","[2248, 2343, 17327, 17151, 9113, 16674, 10473, 20750, 14197, 23774, 24047, 6981, 3849, 985, 4714..."
...,...,...,...,...,...,...,...,...
823,409,"[11874, 9819, 1012, 269, 12867, 15740, 2047, 3778, 339, 14258, 4407, 15442, 3448, 16386, 3360, 1...","[12900, 5099, 10676, 11146, 995, 16476, 4797, 459, 5142, 4905, 6098, 3731, 14978, 9484, 6019, 50...",1,6569,2617,"[14574, 3958, 4831, 11898, 10044, 10559, 18353, 12644, 5366, 2617, 11877, 17813, 12656, 6977, 11...","[3406, 2839, 2999, 43, 5130, 16717, 965, 978, 5886, 3940, 7694, 3289, 5153, 5077, 15897, 2302, 6..."
824,410,"[9772, 12159, 1393, 4054, 12386, 11215, 12893, 10749, 3064, 12890, 6965, 20547, 5359, 9772, 1342...","[4101, 6322, 12421, 10922, 4641, 12294, 4068, 3513, 13046, 13233, 13062, 12821, 13046, 13498, 13...",1,8476,20614,"[8476, 15283, 14503, 4641, 13498, 10922, 13046, 14803, 16514, 16185, 19519, 13979, 13068, 4068, ...","[4054, 12386, 11215, 12893, 10749, 3064, 12890, 6965, 20547, 5359, 9772, 13428, 13554, 13554, 52..."
825,411,"[162, 3999, 10877, 2705, 825, 1129, 2695, 4283, 4623, 4621, 9677, 958, 1052, 1184, 1213, 4583, 3...","[5999, 4084, 11040, 3456, 8912, 919, 3280, 4571, 11211, 3403, 4038, 4042, 4039, 4352, 4351, 4585...",1,4947,16113,"[4063, 2637, 5999, 8643, 4038, 4947, 19543, 8575, 4232, 11509, 3403, 8644, 3148, 7840, 8634, 110...","[12010, 11540, 11540, 8901, 12054, 12171, 12212, 12303, 12194, 12756, 11308, 12431, 12673, 12637..."
826,412,"[14809, 15385, 15917, 14814, 14815, 5210, 13842, 13602, 3940, 16543, 15997, 3334, 14882, 13686, ...","[7195, 7196, 9511, 14449, 4094, 7382, 2339, 13459, 4031, 16302, 3353, 16119, 16544, 14089, 14098...",1,17008,26302,"[20605, 17800, 9511, 15086, 4031, 25004, 26302, 16076, 17008, 26192, 14098, 20101, 25490, 23374,...","[19453, 19203, 19918, 19918, 20130, 20130, 20338, 20338, 20353, 20353, 19491, 20555, 20696, 2216..."


In [62]:
full_sequence_df_new.to_csv(f"../Dataset/{dataset_name}/processed/full_sequence_df.csv", sep="\t", index=False)

In [63]:
def get_pairwise_data(row):
    candidate = row.itemid_candidate
    pos = candidate[row.pos_target_index]
    neg_list = copy.deepcopy(candidate)
    neg_list.remove(pos)
    pair_list = []
    answer_list = []
    for neg in neg_list:
        pair = [pos, neg]
        random.shuffle(pair)
        answer = pair.index(pos)
        pair_list.append(pair)
        answer_list.append(answer)
    
    return pair_list, answer_list

In [68]:
def get_topk_final_data(df):
    for n_candidate in [6,7,8,9,10,15,20,25]:
        for n_history in [5]:
            print(f"n_candidate:{n_candidate} ; n_history{n_history}")
            LLM_top1_data = copy.deepcopy(df)
            LLM_top1_data["itemid_candidate"] = LLM_top1_data.apply(get_sub_candidate, n_candidate=n_candidate, axis=1)
            LLM_top1_data["itemid_history"] = LLM_top1_data.apply(get_sub_history, n_history=n_history, axis=1)
            LLM_top1_data["pos_target_index"] = LLM_top1_data.apply(get_pos_target_index, axis=1)
            LLM_top1_data = LLM_top1_data.sample(frac=1.0, random_state=2023).reset_index(drop=True)
            LLM_top1_data[["pair_itemid_candidate", "pair_pos_target_index"]] = LLM_top1_data.apply(get_pairwise_data, axis=1, result_type="expand")
            LLM_top1_data.to_csv(f"../{dataset_name}/LLM/topk_candidate@{n_candidate}_history@{n_history}.csv", sep="\t", index=False)

In [69]:
get_topk_final_data(full_sequence_df_new)

n_candidate:6 ; n_history5
n_candidate:7 ; n_history5
n_candidate:8 ; n_history5
n_candidate:9 ; n_history5
n_candidate:10 ; n_history5
n_candidate:15 ; n_history5
n_candidate:20 ; n_history5
n_candidate:25 ; n_history5


# 5. Generate Datamaps

In [70]:
def process_name(s):
    s = s.replace("&amp;", "&")
    s = s.replace("  ", " ")
    s = s.replace("\n", " ")
    s = s.replace("\\", " ")
    return s
item_df["title"] = item_df["title"].apply(lambda x: process_name(x))
item_df

,itemid,title,categories,brand,sales_type
0,0,Songs for the Shepherd,'Christian',Keith Green,CDs & Vinyl
1,3,Apocalyptic Love: UK Fan Pack Edition,'Rock',Slash,CDs & Vinyl
2,1,Panic of Girls Fan Pack,"'Alternative Rock', 'New Wave'",Blondie,CDs & Vinyl
3,2,Chickenfoot III: Classic Rock,"'Rock', 'Hard Rock'",Chickenfoot,CDs & Vinyl
4,4,Re-Machined: a Tribute to Machine Head,'Rock',Deep Purple (Various Artists),CDs & Vinyl
...,...,...,...,...,...
26314,26314,All the Singles,'Pop',The Turtles,CDs & Vinyl
26315,26315,Weightless,'Pop',Liz Longley,CDs & Vinyl
26316,26316,Bad Moms Soundtrack,'Pop',Various,CDs & Vinyl
26317,26317,Kinda Don't Care,'Country',Justin Moore,CDs & Vinyl


In [71]:
id2item_dict = item_df.set_index("itemid")["title"].to_dict()
id2item_dict
item2id_dict = item_df.set_index("title")["itemid"].to_dict()
item2id_dict

{0: 'Songs for the Shepherd',
 3: 'Apocalyptic Love: UK Fan Pack Edition',
 1: 'Panic of Girls Fan Pack',
 2: 'Chickenfoot III: Classic Rock',
 4: 'Re-Machined: a Tribute to Machine Head',
 5: 'Live on Air 1980',
 6: 'The Kush',
 7: 'Comatose',
 8: 'Billy Elliot the Musical',
 9: 'Transformed Man',
 10: 'Retrospective Hybrid',
 11: 'Contraband Ed',
 12: 'Cross Purposes - Live',
 13: 'Beyond Belief',
 14: 'Repeat the Sounding Joy',
 15: 'Heat of the Sun',
 16: 'Between Two Worlds',
 18: 'Famous Blue Raincoat',
 22: 'Senor Blues',
 19: 'Dancing the Blues',
 21: 'Leo Kottke Live',
 23: 'Pete',
 17: 'Hunter',
 26: "That's Them",
 20: 'Mystery Lady',
 25: 'All Day, All Night',
 27: 'Never Turn Your Back on a Friend',
 24: 'Changing Faces',
 32: 'Hotel California',
 30: 'Chance',
 34: "Cookin' With the Miles Davis Quintet",
 31: 'Wheels Of Fire',
 29: 'Home',
 35: 'Leon Russell',
 28: 'Smash Your Head Against The Wall',
 38: 'The Golden Age of Underground Radio With Tom Donahue',
 39: 'The T

{'Songs for the Shepherd': 0,
 'Apocalyptic Love: UK Fan Pack Edition': 3,
 'Panic of Girls Fan Pack': 1,
 'Chickenfoot III: Classic Rock': 2,
 'Re-Machined: a Tribute to Machine Head': 4,
 'Live on Air 1980': 5,
 'The Kush': 6,
 'Comatose': 7,
 'Billy Elliot the Musical': 8,
 'Transformed Man': 9,
 'Retrospective Hybrid': 10,
 'Contraband Ed': 11,
 'Cross Purposes - Live': 12,
 'Beyond Belief': 13,
 'Repeat the Sounding Joy': 14,
 'Heat of the Sun': 15,
 'Between Two Worlds': 18765,
 'Famous Blue Raincoat': 18,
 'Senor Blues': 22,
 'Dancing the Blues': 19,
 'Leo Kottke Live': 21,
 'Pete': 23,
 'Hunter': 25367,
 "That's Them": 26,
 'Mystery Lady': 20,
 'All Day, All Night': 25,
 'Never Turn Your Back on a Friend': 27,
 'Changing Faces': 24,
 'Hotel California': 32,
 'Chance': 30,
 "Cookin' With the Miles Davis Quintet": 34,
 'Wheels Of Fire': 31,
 'Home': 26191,
 'Leon Russell': 35,
 'Smash Your Head Against The Wall': 28,
 'The Golden Age of Underground Radio With Tom Donahue': 38,
 '

In [73]:
datamaps = {}
datamaps["id2item_dict"] = id2item_dict
datamaps["item2id_dict"] = item2id_dict

import json
json_str = json.dumps(datamaps)
with open(f"../{dataset_name}/LLM/datamaps.json", 'w') as out:
    out.write(json_str)
    

1746016

In [75]:
item2pop_dict = item_pop.set_index("itemid")["pop"].to_dict()
for i in id2item_dict.keys():
    if i not in item2pop_dict.keys():
        item2pop_dict[i] = 0
item2pop_dict

json_str = json.dumps(item2pop_dict)
with open(f"../{dataset_name}/LLM/popularity_datamaps.json", 'w') as out:
    out.write(json_str)

{25260: 6555,
 25433: 4858,
 25648: 3527,
 25191: 2808,
 25383: 2759,
 19811: 2646,
 19883: 2134,
 21375: 2030,
 23538: 1942,
 25845: 1887,
 23631: 1813,
 2328: 1808,
 5877: 1689,
 14803: 1663,
 25346: 1605,
 4319: 1548,
 1448: 1521,
 24117: 1511,
 13677: 1498,
 15610: 1486,
 10513: 1484,
 2634: 1457,
 15363: 1444,
 23700: 1417,
 2154: 1408,
 12121: 1389,
 5006: 1384,
 22833: 1383,
 12294: 1333,
 13054: 1253,
 19732: 1211,
 3965: 1210,
 3897: 1210,
 23233: 1156,
 2439: 1124,
 17542: 1099,
 2732: 1096,
 26219: 1094,
 14414: 1079,
 4613: 1073,
 4426: 1060,
 13979: 1059,
 14258: 1035,
 17963: 1031,
 18091: 1030,
 24548: 1026,
 15223: 1026,
 22979: 1024,
 869: 1010,
 4427: 1001,
 19918: 985,
 4618: 966,
 5526: 952,
 18760: 949,
 20178: 940,
 11683: 932,
 11547: 932,
 23716: 930,
 25123: 925,
 23679: 924,
 21643: 921,
 23943: 920,
 14379: 904,
 2957: 883,
 20450: 877,
 25487: 874,
 14066: 873,
 17841: 858,
 8764: 857,
 8608: 856,
 288: 853,
 15074: 852,
 23464: 848,
 3931: 841,
 23093: 838,

334694

## Example of Loading Final Evaluation Data 

In [77]:
read_example_df = pd.read_csv(f"../{dataset_name}/LLM/topk_candidate@5_history@5.csv", delimiter="\t")
read_example_df = read_example_df[["userid", "itemid_history", "pos_target_index", "itemid_candidate", "pair_itemid_candidate", "pair_pos_target_index"]]
for col in ['itemid_history', 'itemid_candidate', "pair_itemid_candidate", "pair_pos_target_index"]:
    read_example_df[col] = read_example_df[col].apply(lambda x: eval(x))
    
read_example_df

,userid,itemid_history,pos_target_index,itemid_candidate,pair_itemid_candidate,pair_pos_target_index
0,368,"[27737, 27781, 27790, 27796, 27826]",2,"[28426, 21108, 27850, 16683, 13451]","[[27850, 28426], [27850, 21108], [16683, 27850], [13451, 27850]]","[0, 0, 1, 1]"
1,208,"[4186, 6194, 8744, 11293, 23311]",2,"[20267, 18222, 23529, 2581, 25927]","[[23529, 20267], [23529, 18222], [2581, 23529], [25927, 23529]]","[0, 0, 1, 1]"
2,216,"[26957, 28656, 6368, 3468, 25025]",3,"[30264, 22236, 30606, 4696, 18873]","[[30264, 4696], [4696, 22236], [4696, 30606], [4696, 18873]]","[1, 0, 0, 0]"
3,345,"[4590, 18403, 18497, 19024, 10122]",4,"[24564, 5935, 6284, 18015, 9256]","[[9256, 24564], [9256, 5935], [9256, 6284], [18015, 9256]]","[0, 0, 0, 1]"
4,317,"[5386, 175, 2460, 2415, 6867]",2,"[8835, 19746, 7714, 2169, 25016]","[[8835, 7714], [7714, 19746], [7714, 2169], [7714, 25016]]","[1, 0, 0, 0]"
...,...,...,...,...,...,...
9085,398,"[29512, 3807, 29575, 29657, 22694]",2,"[2767, 19867, 17303, 28289, 29067]","[[2767, 17303], [19867, 17303], [28289, 17303], [29067, 17303]]","[1, 1, 1, 1]"
9086,595,"[6055, 20687, 20256, 14337, 16046]",2,"[16335, 21952, 20674, 12043, 15151]","[[20674, 16335], [21952, 20674], [20674, 12043], [20674, 15151]]","[0, 1, 0, 0]"
9087,16,"[5407, 5813, 10697, 13929, 3645]",2,"[3342, 5437, 15959, 1635, 3534]","[[3342, 15959], [15959, 5437], [1635, 15959], [3534, 15959]]","[1, 0, 1, 1]"
9088,203,"[7728, 11668, 9318, 23837, 896]",3,"[27678, 24409, 22184, 22454, 6852]","[[27678, 22454], [24409, 22454], [22454, 22184], [22454, 6852]]","[1, 1, 0, 0]"


In [79]:
read_example_df = pd.read_csv(f"../{dataset_name}/LLM/topk_candidate@5_history@5.csv", delimiter="\t")
read_example_df = read_example_df[["userid", "pos_target_index", "itemid_candidate"]]
read_example_df["itemid_candidate"] = read_example_df["itemid_candidate"].apply(lambda x: x[1:-1])
candidate = read_example_df['itemid_candidate'].str.split(',', expand=True)
for col in candidate.columns:
    candidate[col] = candidate[col].apply(lambda x: int(x))
candidate.values

array([[28426, 21108, 27850, 16683, 13451],
       [20267, 18222, 23529,  2581, 25927],
       [30264, 22236, 30606,  4696, 18873],
       ...,
       [ 3342,  5437, 15959,  1635,  3534],
       [27678, 24409, 22184, 22454,  6852],
       [26769, 27124, 28331, 27772, 25304]])